In [ ]:
import numpy as np
from configs import ExperimentConfig
from experiment_builder import ExperimentBuilder
from registry_system import Registry, create_registry
from utils import *

# Create the experiment configuration
config = ExperimentConfig(
    time=10.0,  # 10 seconds
    hz=10.0,    # 10 Hz
    trials=100  # 100 trials
)


# Create registries for properties and motion functions
properties_registry = create_registry("field_object_properties")
motion_registry = create_registry("motion_functions")

# Register motion functions in the registry
motion_registry.register("linear", linear, "Linear motion function")
motion_registry.register("elliptical", elliptical, "Elliptical motion function")
motion_registry.register("stationary", lambda **kwargs: None, "Stationary (no motion)")

# Create experiment builder
builder = ExperimentBuilder(
    config=config,
    properties_registry=properties_registry,
    motion_registry=motion_registry
)


# Create motion parameters
linear_params = {
    'start': np.random.normal(0, 1, size=(3, 2)),  # 3 sensors, 2D positions
    'velocity': np.random.uniform(1.0, 2.0, 3),    # Random velocities
    'angle': np.random.uniform(0, 2*np.pi, 3)      # Random angles
}

elliptical_params = {
    'center': np.array([[0.0, 0.0]]),   # Center at origin
    'periods': np.array([1.0]),         # One period over the simulation
    'a': np.array([3.0]),               # Semi-major axis
    'b': np.array([1.5]),               # Semi-minor axis
    'phi': np.array([np.pi/4])          # 45 degrees rotation
}
stationary_params = {
    'start': np.random.normal(0, 0, size=(2, 2)),  # 3 sensors, 2D positions
    'velocity': np.random.uniform(0.0, 0.0, 2),    # Random velocities
    'angle': np.random.uniform(0, 0, 2)      # Random angles
}


# Add field objects to experiment
# Add 3 toy sensors with linear motion
builder.add_objects(
    count=3, 
    object_class="sensor", 
    object_type="toy", 
    motion_type="linear", 
    motion_params=linear_params
)

# Add 1 toy sensor with elliptical motion
builder.add_objects(
    count=1, 
    object_class="sensor", 
    object_type="toy", 
    motion_type="elliptical", 
    motion_params=elliptical_params
)

# Add 1 toy source that is stationary
builder.add_objects(
    count=2, 
    object_class="source", 
    object_type="toy", 
    motion_type="linear", 
    motion_params=stationary_params
)

# Build the experiment
experiment = builder.build()

# Print experiment summary
print(f"Experiment Configuration:")
print(f"- Duration: {config.time} seconds")
print(f"- Sampling rate: {config.hz} Hz")
print(f"- Trials: {config.trials}")
print(f"- Time steps: {config.steps}")
print(f"- Field objects: {len(experiment['field_objects'])}")
for i, obj in enumerate(experiment['field_objects']):
    print(f"  {i+1}. {obj['id']} - {obj['motion_type']} motion")

paths = np.vstack([call_with_filtered(*args) for args in experiment['field_motion']])
compute_all_distances(paths[:-2], paths[-2:]) #(n_src, n_snsr, steps, dims)

Experiment Configuration:
- Duration: 10.0 seconds
- Sampling rate: 10.0 Hz
- Trials: 100
- Time steps: 100
- Field objects: 6
  1. toy_sensor_1 - linear motion
  2. toy_sensor_2 - linear motion
  3. toy_sensor_3 - linear motion
  4. toy_sensor_1 - elliptical motion
  5. toy_source_1 - linear motion
  6. toy_source_2 - linear motion


(2, 4, 101)

2